ETP, exchange traded products, track underlying securities, indexes, or other financial instruments, similarly to stock these are traded on exchanges which implies daily price fluctuations.Each ETP can be include as many underelying assets, digital ones also, as the creator decides, thus, gaining and losing in value in sync with the underlying assets, ETFs are the msot popular types of ETPs. These financial basket type of instruments are a great cost-effective way for institutions and individuals to gain exposure to a specific market, therefore crypto ETPs are great for investors willing to gain an exposure to crypto without the hassle of holding coins on spot or wallets.

The following is an EDA, exploratory data analysis, aiming for insights about:

ETP's liquidity (bid/ask spreads & volume)
Underlying assets' liquidity (bid/ask spreads & volume)
Relevant correlations

In [37]:
import pandas as pd

df = pd.read_csv('ETP Explorer export_splittedintocolumns.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   FundLongName                125 non-null    object 
 1   ValorSymbol                 125 non-null    object 
 2   FundReutersTicker           125 non-null    object 
 3   FundBloombergTicker         125 non-null    object 
 4   ValorNumber                 125 non-null    int64  
 5   ISIN                        125 non-null    object 
 6   IssuerLongNameDesc          125 non-null    object 
 7   IssuerNameFull              125 non-null    object 
 8   TradingBaseCurrency         125 non-null    object 
 9   FundCurrency                125 non-null    object 
 10  ManagementFee               125 non-null    float64
 11  ReplicationMethodDesc       125 non-null    object 
 12  ManagementStyleDesc         125 non-null    object 
 13  MarketMakers                125 non

In [38]:
#Let's look at a preview of our data
df.head()

,FundLongName,ValorSymbol,FundReutersTicker,FundBloombergTicker,ValorNumber,ISIN,IssuerLongNameDesc,IssuerNameFull,TradingBaseCurrency,FundCurrency,...,DailyLowPrice,OnMarketVolume,OffBookVolume,TotalTurnover,TotalTurnoverCHF,ProductLineDesc,AssetClassDesc,UnderlyingGeographicalDesc,LegalStructureCountryDesc,UnderlyingProviderDesc
0,1 FiCAS Active Bitcoin Exchange Traded Product,BTCB,BTCB.S,BTCB SE,113027698,CH1130276988,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 FiCAS Active Ethereum Exchange Traded Product,ETHB,ETHB.S,ETHB SE,113027699,CH1130276996,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,EUR,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,USD,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


If we fetch the data from yahooFinance, we care about the columns 'FundLongName', 'ISIN', 'FundCurrency's,'FundUnderlyingDecription'